In [ ]:
#need to install the following packages using pip or pip3:
# pip3 install beautifulsoup4
# pip3 install numpy
# pip3 install cv2

#Testing

#Actual Functions to change things

In [1]:
#function for reading xml file and return the class from it 
from bs4 import BeautifulSoup as bs
import re
import numpy as np
import cv2
import os
import shutil

def xml_to_class(filepath):
  #<name>quarter</name>
  with open(filepath, 'r') as f:
    data = f.read()

  Bs_data = bs(data, "xml")
  tagline = str(Bs_data.find('name'))
  m = re.match(r"^<name>([A-za-z\d ]+)</name>$", tagline)
  class_val = m.group(1)

  if class_val and class_val == "1 dollar":
    class_val = "dollar"
  
  if class_val:
    return class_val
  else:
    return "Unknown"
# xml_to_class(training_path)
################################################
def rotated(A,angle):
#rotates the point A by the degrees given
#angle = 0-360
  theta = np.radians(angle)
  c, s = np.cos(theta), np.sin(theta)
  R = np.array(((c, -s), (s, c)))
  A = [0,1]
  rotated_A = A @ R.T
  return np.around(rotated_A)
  
  # print(A)
  # print(np.around(rotated_A))
###################################################################################
def filenames_in(source,jpg):
  #if jpg = true, just return the list of jpg files in the folder
  path, dirs, files = next(os.walk(source))
  lst = []
  if jpg:
    for f in files:
      m = re.match(r"^([A-za-z\d]+).jpg$", f)
      if m:
        lst.append(m.group(1))
    return lst

  return files
###################################################################################
def make_folders(folder_names_list,parent_path):
  #folders = ["penny","nickel","dime","quarter","dollar","Unknown"]
  for f in folder_names_list:
    path = parent_path + f
    try:
        os.mkdir(path)
    except OSError:
        print (f"Could not create {path}")
    else:
        print ("Successfully created the directory %s " % path)
###################################################################################
def remove_all_folders(folder_path):
      shutil.rmtree(folder_path)
###################################################################################
##Reorganizing the JPGs into proper label buckets
def folder_to_label_buckets(from_folder,to_parent_folder):
  file_names_list = filenames_in(from_folder,jpg = True)

  new_file_count = 0
  folder = "Unknown"
  for f in file_names_list:
    img_path = f"{from_folder}{f}.jpg"
    xml_path = f"{from_folder}{f}.xml"
    try:
          img = Image.open(img_path) 
    except Exception:
          print(f"folder to label failed to place img: {i}")
    folder = xml_to_class(xml_path)  
    #print(f"img {i} goes in {folder}")
    img_save_path = f"{to_folder}{folder}/{f}.jpg"
    shutil.copyfile(img_path, img_save_path)

#Given a folder, takes all the jpg files and creates 10 additional versions of each and saves them in the folder you choose

In [2]:
#go through all image files in a folder 
from PIL import Image 
import os
import shutil
import numpy as np
from random import seed
from random import random

#if xml is true then it will also copy the xml files wherever the jpg files are
#going
def rot_images(num_times, from_folder, to_folder,xml):

  file_names_list = filenames_in(from_folder,jpg = True)

  folder = "rand"
  new_file_count = 0

  for f in file_names_list:
    img_path = f"{from_folder}{f}.jpg"
    xml_path = f"{from_folder}{f}.xml"  
    try:
          img = Image.open(img_path) 
    except Exception:
          print(f"failed to open img: {i}")

    #copy the original image to the desired folder
    img_save_path = f"{to_folder}IMG_{new_file_count}.jpg"
    shutil.copyfile(img_path, img_save_path)
    if xml:
      #also copy the xml file cooresponding with this image
      xml_save_path = f"{to_folder}IMG_{new_file_count}.xml"
      shutil.copyfile(xml_path,xml_save_path)
    #added 2 new files
    new_file_count += 1

    angles = np.linspace(0,300,num_times)
    angles = np.around(angles)
    #build the angle buckets
    seed()
    for i in range(len(angles)):
      r = round(10*random())
      angles[i] += r

    for x in angles:
      rot_img = img.rotate(x)
      img_save_path = f"{to_folder}IMG_{new_file_count}.jpg"
      rot_img.save(img_save_path)
      if xml:
        #also copy the xml file cooresponding with this image
        xml_save_path = f"{to_folder}IMG_{new_file_count}.xml"
        shutil.copyfile(xml_path, xml_save_path)
        #added 2 new files, jpg and xml
      new_file_count += 1

#Cropping images in a folder

In [4]:
import cv2
import numpy as np
import math

def show_image(image):
    cv2.imshow(image)
    # c = cv2.waitKey()
    # if c >= 0 : return -1
    # return 0

def save_cropped_img(img_path,save_path,write,margin_ratio):
  #if write == true, it will write over the image to save_path
  og_image = cv2.imread(img_path)
  img_gray = cv2.cvtColor(og_image, cv2.COLOR_BGR2GRAY)
  
  sz = 5
  kernel = np.ones((sz,sz),np.float32)/(sz**2)
  img_gray = cv2.filter2D(img_gray,-1,kernel)

  #ret, im = cv2.threshold(img_gray, 100, 255, cv2.THRESH_BINARY_INV)
  ret, thresh_img = cv2.threshold(img_gray, 88, 255, cv2.THRESH_BINARY_INV)
  sz = 5
  kernel = np.ones((sz,sz),np.float32)/(sz**2)
  thresh_img = cv2.filter2D(thresh_img,-1,kernel)

  cnt, hierarchy = cv2.findContours(thresh_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
  #res_img = cv2.drawContours(thresh_img, cnt,-1, (255,255,255), 2)
  #get the largest contour by area
  mx = 0
  largest_cnt = 0
  for c in cnt:
    area = cv2.contourArea(c)
    if mx < area:
      mx = area
      largest_cnt = c

  #x,y is the top coordinate
  x,y,w,h = cv2.boundingRect(largest_cnt)
  
  #scale the box size a bit bigger
  r = math.sqrt(w**2 + h**2)
  margin = int(r*margin_ratio)
  # print(f"r = {r} margin = {margin}")
  # print(f"x = {x} y = {y} w = {w} h = {h}")
  w = w+margin
  h = h+margin
  #1 - 2
  #3 - 4
  im_height_index = og_image.shape[0] - 1
  im_width_index = og_image.shape[1] - 1

  xabove = max(x - margin,0)
  xbelow = min(x+w,im_width_index)

  yabove = max(y - margin,0)
  ybelow = min(y+h,im_height_index)

  box_rows_cols = (yabove,ybelow, xabove,xbelow)
  #print(f"x_above = {xabove}\n x_below = {xbelow}\n y_below = {ybelow}\n y_above = {yabove}\n")
  #this actually draws a bounding box around the coin
  #res_img = cv2.rectangle(thresh_img,(xabove,yabove),(xbelow,ybelow),(255,255,255),5)
  cropped_image = og_image[yabove:ybelow, xabove:xbelow]
  #cv2_imshow(cropped_image)
  #cv2_imshow(cropped_image)
  if write:
    cv2.imwrite(save_path, cropped_image)
  return cropped_image

################################################
def crop_all_images_in_folder(from_folder, to_folder,xml,margin_ratio):
#if xml == True, then copy the xml files to that folder as well
#margin ratio of about .4 is good
  file_names_list = filenames_in(from_folder,jpg = True)
  for f in file_names_list:
    img_path = f"{from_folder}{f}.jpg" 
    #copy the original image to the desired folder
    img_save_path = f"{to_folder}{f}.jpg"
    cropped = save_cropped_img(img_path,img_save_path,write = True, margin_ratio = .3)
    if xml == True:
      xml_path = f"{from_folder}{f}.xml"
      xml_save_path = f"{to_folder}{f}.xml"
      shutil.copyfile(xml_path, xml_save_path)


#Set up directory structure
need to be in the same folder as Coin_Images_labels

In [23]:
#if you want to delete new_dir and start over 
dot_slash = "./"
new_dir_path = dot_slash + "new_dir/"
remove_all_folders(new_dir_path)

In [25]:
#Set Up proper directory structure
#need to be in the same folder as Coin_Images_labels
#Create One folder where everything is located

dot_slash = "./"

new_dir_path = dot_slash + "new_dir/"

#if you want to recursively delete new_dir, run: remove_all_folders(new_dir_path)
if not os.path.isdir(new_dir_path):
  os.mkdir(new_dir_path)

#remake directory structure:
base = "test"
#top_level is the folder where we will do everything
top_level = f"{new_dir_path}{base}_dir/"
#first remove all files in new_dir if there are any
if os.path.isdir(top_level):
  # delete all folders in the top_level directory
  remove_all_folders(top_level)
else:
  os.mkdir(top_level)

#create main_folders
dir_folders = [f"{base}_rot",f"{base}_cropped",f"{base}_folders"]
make_folders(dir_folders,top_level)

#create label_folders
folders = ["penny","nickel","dime","quarter","dollar","Unknown"]
label_folders_parent = f"{top_level}{base}_folders/"

make_folders(folders,label_folders_parent)


Successfully created the directory /content/drive/MyDrive/ColabNotebooks/Coins/new_dir/test_dir/test_rot 
Successfully created the directory /content/drive/MyDrive/ColabNotebooks/Coins/new_dir/test_dir/test_cropped 
Successfully created the directory /content/drive/MyDrive/ColabNotebooks/Coins/new_dir/test_dir/test_folders 
Successfully created the directory /content/drive/MyDrive/ColabNotebooks/Coins/new_dir/test_dir/test_folders/penny 
Successfully created the directory /content/drive/MyDrive/ColabNotebooks/Coins/new_dir/test_dir/test_folders/nickel 
Successfully created the directory /content/drive/MyDrive/ColabNotebooks/Coins/new_dir/test_dir/test_folders/dime 
Successfully created the directory /content/drive/MyDrive/ColabNotebooks/Coins/new_dir/test_dir/test_folders/quarter 
Successfully created the directory /content/drive/MyDrive/ColabNotebooks/Coins/new_dir/test_dir/test_folders/dollar 
Successfully created the directory /content/drive/MyDrive/ColabNotebooks/Coins/new_dir/test

Need to be in the same folder as Coin_Images_labels
This is how the images get transfered over after transformations

In [ ]:
#need to be in the same folder as Coin_Images_labels
# use Coin_Images_Labels/test/train => 
#   => test_cropped/train_cropped
#     => test_rot/train_rot  
#       => test_folders/train_folders

base = "test"

dot_slash = "./"

new_dir_path = dot_slash + "new_dir/"
#all the files will go into the top_level folder = 
top_level = f"{new_dir_path}{base}_dir/"
#the source of the pics along with xml files 
pics_source = f"{dot_slash}Coin_Images_Labels/{base}/"

#pics source => crop => rotate/create more => and then bucket

#---------   Crop Images   ---------------#
from_folder = f"{pics_source}"
to_cropped_folder = f"{top_level}{base}_cropped/"

print(f"Cropping Images in {from_folder} and placing them in {to_cropped_folder}\n")
#xml = true means that it will copy over the xml files as well
#margin_ratio = margin of how big the box will be 
# as a ratio of the bounding box's hypotenuse
crop_all_images_in_folder(from_folder, to_cropped_folder,xml = True,margin_ratio = .4)

#---------   Rotate  Images   ---------------#
from_folder = to_cropped_folder
to_rot_folder = f"{top_level}{base}_rot/"

print(f"Rotating Images in {from_folder} and placing them in {to_rot_folder}\n")
#xml = true means that it will copy the xml files over as well
rot_images(10, from_folder, to_rot_folder,xml=True)

#---------   Bucket Images   ---------------#
#parent folder has label folders in it
from_folder = to_rot_folder
to_folder = f"{top_level}{base}_folders/"
print(f"Bucketting Images in {from_folder} and placing them in {to_folder}")
folder_to_label_buckets(from_folder,to_folder)

